<a href="https://colab.research.google.com/github/RVijayChiranjithReddy/Fake_face_detection/blob/main/Triple_loss_fake_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.models import *
from keras.optimizers import *
import keras.backend as K
import dlib
from PIL import Image
from skimage import io
import keras

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
	
from keras.applications.vgg16 import VGG16
model = VGG16(weights=None)
trip = Model(model.input,model.layers[-5].output)

In [ ]:
def contrast():
  #inputs = Input(shape = (128,128,3))
  new_input = Input(shape=(224, 224, 3))
  #model = VGG16(include_top=False, input_tensor=new_input, pooling='avg')
  x = trip(new_input)
  layer12 = LeakyReLU(alpha=0.3)(x)
  layer13 = Conv2D(512,strides = 1,kernel_size=(3,3),padding="same")(layer12)
  layer5 = LeakyReLU(alpha=0.3)(layer13)
  layer6 = Flatten()(layer5)
  layer7 = Dense(4096,activation='relu')(layer6)
  layer7 = Dense(4096,activation='relu')(layer7)
  return Model(inputs = new_input,outputs = layer7)

nmodel = contrast()
nmodel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
model (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 7, 7, 512)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 7, 7, 512)         2359808   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 512)         0         
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 4096)              1027

In [ ]:
def triple():
  inputs = Input(shape = (224,224,9))
  r_i = Lambda(lambda x: x[:,:,:,0:3], name='r_i')(inputs)
  f_i = Lambda(lambda x: x[:,:,:,3:6], name='f_i')(inputs)
  r_j = Lambda(lambda x: x[:,:,:,6:], name='r_j')(inputs)
  r_i = nmodel(r_i)
  r_i = Lambda(lambda  x: K.l2_normalize(x,axis=1))(r_i)
  f_i = nmodel(f_i)
  f_i = Lambda(lambda  x: K.l2_normalize(x,axis=1))(f_i)
  r_j = nmodel(r_j)
  r_j = Lambda(lambda  x: K.l2_normalize(x,axis=1))(r_j)
  merge_one = concatenate([r_i, f_i],axis=0)
  merge_two = concatenate([merge_one, r_j],axis=0)
  emb = Dense(4096)(merge_two)
  l2_norm_final = Lambda(lambda  x: K.l2_normalize(x,axis=1))(emb)
  return Model(inputs = inputs,outputs = l2_norm_final)
triple_loss = triple()
triple_loss.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 9) 0                                            
__________________________________________________________________________________________________
r_i (Lambda)                    (None, 224, 224, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
f_i (Lambda)                    (None, 224, 224, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
model_1 (Functional)            (None, 4096)         136620352   r_i[0][0]                        
                                                                 f_i[0][0]                  

In [ ]:
_EPSILON = K.epsilon()
def _loss_tensor(y_true, y_pred):
  # clip to prevent NaN and Inf
  #y_pred = K.clip(y_pred, _EPSILON, 1.0-_EPSILON)
  loss =  tf.convert_to_tensor(0,dtype=tf.float32)
  g = tf.constant(1.0, shape=[1], dtype=tf.float32)
  anchor = y_pred[0,:]
  positive =  y_pred[1,:]
  negative = y_pred[2,:]
  positive_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), -1)
  negative_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)), -1)
  loss_1 = tf.add(tf.subtract(positive_dist, negative_dist), 0.3)
  loss = tf.reduce_sum(tf.maximum(loss_1, 0.0))
  return loss

In [ ]:
triple_loss.compile(loss=_loss_tensor,
 optimizer=SGD(momentum = 0.9,lr=0.001),
 metrics=['accuracy'])

In [ ]:
import cv2
import glob
from random import randint, shuffle
real = glob.glob(r'/content/gdrive/MyDrive/Minor_project/dataset/real/*')
fake = glob.glob(r'/content/gdrive/MyDrive/Minor_project/dataset/fake/*')
print(len(real))
def read_img(fn,is_cropped = False):
  im = cv2.imread(fn)
  im = cv2.cvtColor(im, cv2.COLOR_BGR2LAB)
  if is_cropped:
    face_detector = dlib.get_frontal_face_detector()
    detected_faces = face_detector(im, 1)
    face_frames = [(x.left(), x.top(),x.right(), x.bottom()) for x in detected_faces]
    face = Image.fromarray(im).crop(face_frames[0])
    im = cv2.resize(face,(224,224),interpolation=cv2.INTER_LINEAR)
  else:
    im = cv2.resize(im, (224,224), interpolation=cv2.INTER_CUBIC)
  im = np.array(im)/255*2-1
  return im


16361


In [ ]:
losses = []
accuracies = []
iteration_checkpoints = []
def train(iterations,sample_interval):
  for i in range(iterations):
    ridx = np.random.randint(0, len(real))
    rjdx = np.random.randint(0, len(real)) 
    while ridx == rjdx:
      rjdx = np.random.randint(0, len(real))
    fidx = np.random.randint(0, len(fake))
    fjdx = np.random.randint(0, len(fake)) 
    while fidx == fjdx:
      fjdx = np.random.randint(0, len(fake))
    r_i = read_img(real[ridx])
    r_j = read_img(real[rjdx])
    f_i = read_img(fake[fidx])
    f_j = read_img(fake[fjdx])
    rij = np.concatenate([r_i,r_j,f_i],axis=-1)
    fij = np.concatenate([f_i,f_j,r_i],axis=-1)
    rfii = np.concatenate([r_i,r_j,f_j],axis=-1)
    ffjj = np.concatenate([f_i,f_j,r_j],axis=-1)
    #z = np.zeros((1,1))
    one = np.ones((1,1))
    rij = np.expand_dims(rij,axis=0)
    fij = np.expand_dims(fij,axis=0)
    rfii = np.expand_dims(rfii,axis=0)
    ffjj = np.expand_dims(ffjj,axis=0)
    lossrr,accr= triple_loss.train_on_batch(rij,one)
    lossff,accf= triple_loss.train_on_batch(fij,one)
    lossrf,racc = triple_loss.train_on_batch(rfii,one)
    lossfr,facc = triple_loss.train_on_batch(ffjj,one)
    if (i + 1) % sample_interval == 0:
      #losses.append((lossrr, lossff,lossrf)) 
      #accuracies.append((racc,facc,accf,accr))
      #iteration_checkpoints.append(i + 1)
      #print("%d[lossrf: %f, acc.: %.2f%%] [lossrf: %f, acc.: %.2f%%]" %(i + 1,lossrf,racc,lossfr,facc))
      print("%d [lossrr: %f] [lossff: %f] [lossff: %f] [lossff: %f]" %(i + 1, lossrr,lossff,lossfr,lossrf))

In [ ]:
train(5000,50)

50 [lossrr: 0.300865] [lossff: 0.300303] [lossff: 0.299597] [lossff: 0.300462]
100 [lossrr: 0.299855] [lossff: 0.300117] [lossff: 0.299828] [lossff: 0.300074]
150 [lossrr: 0.300498] [lossff: 0.300114] [lossff: 0.299506] [lossff: 0.300651]
200 [lossrr: 0.300127] [lossff: 0.299293] [lossff: 0.299921] [lossff: 0.300572]
250 [lossrr: 0.299925] [lossff: 0.299770] [lossff: 0.300288] [lossff: 0.300272]
300 [lossrr: 0.300042] [lossff: 0.300271] [lossff: 0.300113] [lossff: 0.300208]
350 [lossrr: 0.300096] [lossff: 0.300214] [lossff: 0.300499] [lossff: 0.299843]
400 [lossrr: 0.299828] [lossff: 0.299911] [lossff: 0.300115] [lossff: 0.299674]
450 [lossrr: 0.300104] [lossff: 0.299575] [lossff: 0.299529] [lossff: 0.299932]
500 [lossrr: 0.299724] [lossff: 0.299819] [lossff: 0.299827] [lossff: 0.299776]
550 [lossrr: 0.299918] [lossff: 0.299929] [lossff: 0.299945] [lossff: 0.299925]
600 [lossrr: 0.300028] [lossff: 0.300405] [lossff: 0.300376] [lossff: 0.299544]
650 [lossrr: 0.300220] [lossff: 0.300360]

In [ ]:
def hello():
  inputs = Input(shape=(224,224,6))
  r_i = nmodel(inputs[:,:,:,0:3])
  r_j = nmodel(inputs[:,:,:,3:])
  score = tf.reduce_sum(tf.square(tf.subtract(r_i, r_j)), -1)
  outputs = Activation('sigmoid')(score)
  return Model(inputs=inputs,outputs = outputs)
embed = hello()


In [ ]:
fk = read_img('/content/testfake.PNG')
fk = np.expand_dims(fk,axis=0)
for i in range(20):
  ridx = np.random.randint(0, len(fake))
  r_i = read_img(fake[ridx])
  rij = np.expand_dims(r_i,axis=0)
  rij = np.concatenate([rij,fk],axis=-1)
  print(embed.predict(rij))

NameError: ignored